*bucket2 file takes train/test/val into account*

In [1]:
import os
from google.cloud import storage
from PIL import Image
from io import BytesIO
import pandas as pd

In [2]:
client = storage.Client(project='prime-basis-374911')
bucket = client.get_bucket('kitchen-ai-images')


In [3]:
#path_to_dataset = os.path.join("..","..","data")
#os.listdir("../..")
#path_to_dataset

In [4]:
#ideas include updating to allow for getting specific class
#or stopping at specific class
#or loading n of each class
#set classes to ignore

##SKIPPED MUSHROOM 21
#filenames_to_skip = ["mushroom_21.jpg"]

#set to empty list just after, used for jumping in after image error
classes_to_skip = [
'asparagus',
'beetroot',
'broccoli',
'cabbage',
'capsicum',
'carrot',
'cauliflower',
'celery',
'corn',
'cucumber',
'eggplant',
'garlic',
'ginger',
'leek',
'lettuce']
classes_to_skip = []



mini_set_stop_class = "capsicum"

#used to correctly index incoming data from 2 sets
multi_set_input_class = ["spinach"]

In [5]:
def save_bucket_data_to(miniset=False):
    all_files = bucket.list_blobs()
    
    #set path to dataset
    # if miniset:
    #     path_to_dataset = os.path.join("..", "test_data", "0_test_miniset")
    # else:
    #     path_to_dataset = os.path.join("..", "data")
    
    #path_to_dataset = os.path.join("..", "test_data", "1_test_split_set", "test")
    
    #path_to_data_folder = os.path.join("..", "..", "test_data", "1_second_load_ready_for_spinach")
    path_to_data_folder = os.path.join("..", "..", "data")#, "1_second_load_ready_for_spinach")
    
    #class_limit = 50
    class_count = 0
    train_count = 0
    test_count = 0
    val_count = 0
    
    current_class = ""
    #this_file = next(all_files)
    #for i in range(10):
    
    for this_file in all_files:
        und_split = this_file.name.split('_')
        dot_split = this_file.name.split('.')
        food_bucket_path = this_file.path #to track old names

        #get info from bucket path name
        food_class = und_split[0].lower()
        file_ext = dot_split[1] #print(img_ext)
        file_num = dot_split[0].split('_')[-1] #print(file_num)
        
        #data_split = "train" "test" "val"
        data_split = dot_split[0].split('_')[-2]      
        
        #make sure the folder is called val
        if data_split == "validation":
            data_split = "val"
        
        #create path to correct folder
        path_to_dataset = os.path.join(path_to_data_folder, data_split)

        #tracking
        if food_class != current_class:
            #if statement avoids printing empty for first
            if current_class:
                print(f"loaded total {class_count} of {current_class}")
                print(f"{train_count} train")
                print(f"{val_count} val")
                print(f"{test_count} test")
            
            print(f"loading {food_class}...")
            current_class = food_class
            class_count = 0
            train_count, test_count, val_count = 0,0,0
                        
        #create new file path
        new_filename = food_class + "_" + str(class_count) + ".jpg" #"." + file_ext
        new_dir_path = os.path.join(path_to_dataset, food_class) #used to create folder if not already existing
        new_path = os.path.join(new_dir_path,new_filename)   #class/class-n.ext
        
        ## STOPPING / SKIPPING CONDITIONS
        
        #STOP AT GARLIC FOR MINISET
        #if miniset and (food_class == mini_set_stop_class):
        #    break
        
        if food_class in classes_to_skip:
            continue
        #if new_filename in filenames_to_skip:
        #    continue
        #if class_count > class_limit:
        #    continue
            
        try:
            #get image from bucket
            file_image = Image.open(BytesIO(this_file.download_as_bytes()))
            #convert to rgb ==> jpg
            rgb_image = file_image.convert('RGB')

            #save image
            os.makedirs(new_dir_path, exist_ok=True) #True means don't create new dir if already exists #os.makedirs(path, exist_ok=True)
            test_img = rgb_image.save(new_path)
            
            #if test_img.verify():
            class_count += 1
            
            if data_split == "train":
                train_count += 1
            if data_split == "test":
                test_count += 1
            if data_split == "val":
                val_count += 1
            
        except:
            print(f"error loading from {food_bucket_path} \n to {new_path}")
    
    num_files = sum(1 for _ in all_files)
    
    print(f"Done! loaded {len(all_files)}")

In [6]:
#save_bucket_data_to()

In [7]:
save_bucket_data_to()

loading asparagus...
loaded total 60 of asparagus
42 train
9 val
9 test
loading beetroot...
error loading from /b/kitchen-ai-images/o/beetroot_ir_test_4.jpg 
 to ../../data/test/beetroot/beetroot_5.jpg


/opt/conda/lib/python3.7/site-packages/PIL/Image.py:997: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


error loading from /b/kitchen-ai-images/o/beetroot_ir_train_23.jpg 
 to ../../data/train/beetroot/beetroot_26.jpg
error loading from /b/kitchen-ai-images/o/beetroot_ir_validation_4.jpg 
 to ../../data/val/beetroot/beetroot_103.jpg
loaded total 108 of beetroot
88 train
10 val
10 test
loading broccoli...
loaded total 1400 of broccoli
1000 train
200 val
200 test
loading cabbage...
loaded total 1400 of cabbage
1000 train
200 val
200 test
loading carrot...
loaded total 1400 of carrot
1000 train
200 val
200 test
loading cauliflower...
loaded total 1400 of cauliflower
1000 train
200 val
200 test
loading celery...
loaded total 43 of celery
30 train
6 val
7 test
loading corn...
error loading from /b/kitchen-ai-images/o/corn_ir_test_4.jpg 
 to ../../data/test/corn/corn_5.jpg
error loading from /b/kitchen-ai-images/o/corn_ir_train_23.jpg 
 to ../../data/train/corn/corn_26.jpg
error loading from /b/kitchen-ai-images/o/corn_ir_validation_4.jpg 
 to ../../data/val/corn/corn_102.jpg
loaded total 107 

ValueError: ('Iterator has already started', <google.api_core.page_iterator.HTTPIterator object at 0x7f27b211c6d0>)

In [23]:

import os
import pandas as pd

def count_data():
    path_to_data_folder = os.path.join("..", "..", "data")
    path_to_train = os.path.join(path_to_data_folder, "train")
    
    classes_list = os.listdir(path_to_train)
    print(len(classes_list))
    
    cols = ["train","val", "test"]
    out_df = pd.DataFrame(index = classes_list, columns = cols)
    
    egg_dict = {"train" : 1 , "val" : 2 , "test"  :3}
    out_df.loc["cauliflower"]["train"] = 1
    
    for food in classes_list:
        for split in cols:
            path = os.path.join(path_to_data_folder, split, food)
            #print(path)
            count = len(os.listdir(path))
            #print(count)
            
            out_df.loc[food][split] = count
    
    return out_df
    
    
data_counts_df = count_data()
data_counts_df

20


,train,val,test
eggplant,708,151,153
cauliflower,1000,200,200
mushroom,45,9,11
beetroot,2966,10,349
potato,1000,200,200
celery,30,6,7
corn,87,10,10
broccoli,1000,200,200
lettuce,97,9,10
leek,42,9,9
